In [23]:
import numpy as np
import cv2
from tqdm import tqdm

In [24]:
def census(left_img, right_img, M, dMax) :
    win_half = int(M / 2 )
    border = dMax + win_half

    '''pad the images to prevent data loss'''
    imgL = np.pad(left_img, border)
    imgR = np.pad(right_img, border)
    width = imgL.shape[1]
    height = imgL.shape[0]
    dMap = np.zeros_like(imgL)

    '''double for loop to raster scan image'''
    for left_row in tqdm(range(border, height - border)):
        for left_col in range(width - border - 1, border -1, -1):
            costVec = []
            windowL = imgL[left_row - win_half : left_row + win_half + 1, \
                           left_col - win_half : left_col + win_half + 1]
            binWinL = np.ravel((windowL > imgL [left_row , left_col]) * 1)

            '''searching through disparity values to identify minima'''
            for d in range(dMax + 1):
                rowR = left_row
                colR = left_col - d
                windowR = imgR[rowR - win_half : rowR + win_half + 1 , \
                          colR - win_half : colR + win_half + 1]
                binWinR = np.ravel((windowR > imgR[rowR , colR]) * 1)
                costVec.append(sum(binWinL ^ binWinR))
            dMap[left_row, left_col] = np.argmin(costVec)
    return dMap.astype(np.uint8)

In [25]:
# Read in the input images
left_image = cv2.imread('/Users/wang3450/Desktop/ECE661/HW09/input_images/im2.png',0)
right_image = cv2.imread('/Users/wang3450/Desktop/ECE661/HW09/input_images/im6.png',0)

# Hyper Parameters
M = 10
dmax = 52

# apply census transform
dmap = census(left_image, right_image, M, dmax)
dmap_viewable = (dmap / np.max(dmap) * 255).astype(np.uint8)

100%|██████████| 375/375 [03:05<00:00,  2.02it/s]


In [26]:
dmap_gt = cv2.imread('/Users/wang3450/Desktop/ECE661/HW09/input_images/disp2.png')
dmap_gt = cv2.cvtColor(dmap_gt, cv2.COLOR_BGR2GRAY)
dmap_gt = dmap_gt.astype(np.float32) / 4
dmap_gt = dmap_gt.astype(np.uint8)

# compute the error
dmap_gt = np.pad(dmap_gt, (52 + (int(M/2))))
error = abs(dmap.astype(np.int16) - dmap_gt.astype(np.int16)).astype(np.uint8)
print(f'Error Min/Max: {np.min(error)}, {np.max(error)} ')

c1 = (error <= 2) * 255
c1 = c1.astype(np.uint8)

validMask = cv2.imread('/Users/wang3450/Desktop/ECE661/HW09/input_images/occl.png', 0)
validMask = np.pad(validMask, (52 + (int(M/2))))
N = cv2.countNonZero(validMask)

validError = cv2.bitwise_and(validMask, c1)

d2Error = cv2.countNonZero(validError) / N

submit_image = np.hstack((dmap_viewable, validError))
cv2.imwrite('/Users/wang3450/Desktop/ECE661/HW09/results/task3_results/submit_image_' + str(dmax) + "_" + str(M) + '.jpg', submit_image)
print(d2Error)

Error Min/Max: 0, 52 
0.802141536460979
